In [1]:
#

In [2]:
import pandas as pd
import regex as re
import os
from pathlib import Path

In [3]:
df = pd.read_csv('./file_lifecycle_events.csv', encoding='utf-8')
df.columns

Index(['filepath', 'event_type', 'event_datetime', 'datetime_last_modified',
       'commit_hash', 'author_name', 'author_email', 'commit_subject',
       'additions', 'deletions', 'is_binary', 'old_path', 'new_path',
       'source_path'],
      dtype='object')

In [4]:
df

,filepath,event_type,event_datetime,datetime_last_modified,commit_hash,author_name,author_email,commit_subject,additions,deletions,is_binary,old_path,new_path,source_path
0,.codesandbox/Dockerfile,added,2023-08-01T00:47:41,2023-08-01T00:47:41,50450a7dabc794a04e553bf461ab03d19bd1298e,Ives van Hoorne,Ives.v.h@gmail.com,chore: update required node version to 18 (#6837),5.0,0.0,False,NaN,NaN,NaN
1,.codesandbox/tasks.json,added,2022-08-29T16:22:04,2023-08-01T00:47:41,59a1d192d268f403c3d946c52f28960a0045c357,Ives van Hoorne,Ives.v.h@gmail.com,chore: update CodeSandbox links and add a conf...,43.0,0.0,False,NaN,NaN,NaN
2,.codesandbox/tasks.json,modified,2023-08-01T00:47:41,2023-08-01T00:47:41,50450a7dabc794a04e553bf461ab03d19bd1298e,Ives van Hoorne,Ives.v.h@gmail.com,chore: update required node version to 18 (#6837),9.0,2.0,False,NaN,NaN,NaN
3,.dockerignore,added,2020-04-29T08:53:07,2024-08-06T14:21:20,4369d88898297201c1b7d173c06bc1cc26238b54,Niko Heikkilä,yo@nikoheikkila.fi,feat: add option for self-hosting via Docker (...,7.0,0.0,False,NaN,NaN,NaN
4,.dockerignore,modified,2020-07-15T14:25:42,2024-08-06T14:21:20,f9793835e6d7e6df9bfc1be17485c8f911e4a3f8,Minh Nguyen,minhnguyenxx@gmail.com,Restore missing env vars in Docker image (#1922),1.0,0.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26816,yarn.lock,modified,2025-12-08T12:45:26,2026-01-10T20:15:14,bfa4941ad1d95b749c88adba5181597b36ec1ed5,Mark Tolmacs,mark@lazycat.hu,fix: More robust mobile and tablet detection,5.0,0.0,False,NaN,NaN,NaN
26817,yarn.lock,modified,2026-01-02T13:08:27,2026-01-10T20:15:14,0586fc138cc3e1ed893d5204af78a13a352fffc6,kish dizon,kishddizon@gmail.com,feat: add qr code to live session share dialog...,5.0,0.0,False,NaN,NaN,NaN
26818,yarn.lock,modified,2026-01-04T17:16:35,2026-01-10T20:15:14,63e1148280163a36326b7efb5fdfeefebabdb3e4,David Luzar,5153846+dwelle@users.noreply.github.com,feat: stop using CSS filters for dark mode (st...,10.0,0.0,False,NaN,NaN,NaN
26819,yarn.lock,modified,2026-01-05T13:25:40,2026-01-10T20:15:14,9616e63e230489eb165884855655a8a6f6c087c0,Mark Tolmacs,mark@lazycat.hu,feat: Ovoids,13.0,0.0,False,NaN,NaN,NaN


In [5]:
df = df.sort_values(['event_datetime']).reset_index(drop=True)

In [6]:
set(df['event_type'])

{'added', 'copied', 'deleted', 'modified', 'renamed'}

In [7]:
len(set(df['filepath']))

1580

In [9]:
df['event_type'].unique()

array(['added', 'deleted', 'modified', 'renamed', 'copied'], dtype=object)

In [10]:
df

,filepath,event_type,event_datetime,datetime_last_modified,commit_hash,author_name,author_email,commit_subject,additions,deletions,is_binary,old_path,new_path,source_path
0,package.json,added,2020-01-02T04:04:47,2025-12-02T00:37:42,6278cd9366ff3468c1c7f7adf78ccb21fb8a861f,Christopher Chedeau,vjeuxx@gmail.com,Initial commit,28.0,0.0,False,NaN,NaN,NaN
1,public/FG_Virgil.ttf,added,2020-01-02T04:04:47,2024-07-25T19:55:55,6278cd9366ff3468c1c7f7adf78ccb21fb8a861f,Christopher Chedeau,vjeuxx@gmail.com,Initial commit,0.0,0.0,True,NaN,NaN,NaN
2,src/styles.css,added,2020-01-02T04:04:47,2020-01-04T21:14:12,6278cd9366ff3468c1c7f7adf78ccb21fb8a861f,Christopher Chedeau,vjeuxx@gmail.com,Initial commit,5.0,0.0,False,NaN,NaN,NaN
3,README.md,deleted,2020-01-02T04:04:47,2025-12-01T14:15:29,6278cd9366ff3468c1c7f7adf78ccb21fb8a861f,Christopher Chedeau,vjeuxx@gmail.com,Initial commit,NaN,NaN,NaN,NaN,NaN,NaN
4,src/index.js,added,2020-01-02T04:04:47,2020-01-02T22:03:48,6278cd9366ff3468c1c7f7adf78ccb21fb8a861f,Christopher Chedeau,vjeuxx@gmail.com,Initial commit,187.0,0.0,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26816,src/components/PublishLibrary.tsx,modified,2026-01-10T20:15:14,2026-01-10T20:15:14,b9a255407fc5e68893016de56057614da26df50c,David Luzar,5153846+dwelle@users.noreply.github.com,refactor: update SCSS syntax & remove open-col...,4.0,5.0,False,NaN,NaN,NaN
26817,src/components/RadioGroup.scss,modified,2026-01-10T20:15:14,2026-01-10T20:15:14,b9a255407fc5e68893016de56057614da26df50c,David Luzar,5153846+dwelle@users.noreply.github.com,refactor: update SCSS syntax & remove open-col...,1.0,1.0,False,NaN,NaN,NaN
26818,src/components/ShareableLinkDialog.scss,modified,2026-01-10T20:15:14,2026-01-10T20:15:14,b9a255407fc5e68893016de56057614da26df50c,David Luzar,5153846+dwelle@users.noreply.github.com,refactor: update SCSS syntax & remove open-col...,1.0,1.0,False,NaN,NaN,NaN
26819,src/components/Modal.css,modified,2026-01-10T20:15:14,2026-01-10T20:15:14,b9a255407fc5e68893016de56057614da26df50c,David Luzar,5153846+dwelle@users.noreply.github.com,refactor: update SCSS syntax & remove open-col...,1.0,1.0,False,NaN,NaN,NaN
